# Tkinter code for gui

In [ ]:
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
import pandas as pd
import pickle
from tkinter import messagebox
import pickle
with open('trained_model.pkl', 'rb') as file:
       model = pickle.load(file)

class FraudDetectionApp:
    def __init__(self, root):
        self.root = root
        root.title("Transaction Fraud Detection")
        root.geometry("1000x1000")  
        
        self.upload_button = tk.Button(root, text="Upload CSV file", command=self.upload_file, font=("Arial", 12), bg="blue", fg="white")
        self.upload_button.pack(pady=10)

        self.result_frame = ttk.Frame(root)
        self.result_frame.pack(padx=10, pady=10)

        self.result_text = tk.Text(self.result_frame, height=20, width=100, font=("Arial", 12))
        self.result_text.pack(padx=10, pady=10)
        self.result_text.config(bg="lightgray")

    def mark_fraudulent(self, transactions):
        try:
            df = pd.read_csv(transactions)
            if df.empty:
                return "No transactions found in the uploaded CSV."
            predictions = []
            for index, row in df.iterrows():
                step = row['step']
                type = row['type']
                amount = row['amount']
                oldbalanceOrg = row['oldbalanceOrg']
                newbalanceOrig = row['newbalanceOrig']
                oldbalanceDest = row['oldbalanceDest']
                newbalanceDest = row['newbalanceDest']
                var = abs(abs(oldbalanceOrg - newbalanceOrig) + abs(oldbalanceDest - newbalanceDest) - amount)
                type_mapping = {"CASH_OUT": 0, "PAYMENT": 1, "CASH_IN": 2, "TRANSFER": 3, "DEBIT": 4}
                type_numeric = type_mapping[type]
                result = model.predict([[step, type_numeric, amount, oldbalanceOrg, newbalanceOrig, oldbalanceDest, newbalanceDest, var]])
                predictions.append("Fraudulent" if result == "yes" else "Not Fraudulent")

            df['Prediction'] = predictions
            return df.to_string(index=False)

        except Exception as e:
            return f"Prediction error: {str(e)}"

    def upload_file(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.result_text.delete("1.0", tk.END)
            result = self.mark_fraudulent(file_path)
            self.result_text.insert("1.0", result)

class LoginApp:
    def __init__(self, root):
        self.root = root
        root.title("Login")
        root.geometry("300x300")
        
        self.username_label = tk.Label(root, text="Username:")
        self.username_label.pack(pady=10)
        self.username_entry = tk.Entry(root)
        self.username_entry.pack()

        self.password_label = tk.Label(root, text="Password:")
        self.password_label.pack()
        self.password_entry = tk.Entry(root, show="*")
        self.password_entry.pack()

        self.login_button = tk.Button(root, text="Login", command=self.login)
        self.login_button.pack(pady=10)

    def login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
        
        if username == "namagiri" and password == "pass":
            self.root.destroy()
            app_root = tk.Tk()
            app = FraudDetectionApp(app_root)
            app_root.mainloop()
        else:
            messagebox.showerror("Login Failed", "Invalid username or password")

if __name__ == "__main__":
    login_root = tk.Tk()
    login = LoginApp(login_root)
    login_root.mainloop()
